In [777]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.spatial.distance
from sklearn.neighbors import kneighbors_graph

# Load Data

In [778]:
#training_data = pd.read_csv("csv_files/features_1_training_set_1.csv", converters={"word_tok": lambda x: x.strip("[]").replace("'","").split(", ")})
#training_data = pd.read_csv("csv_files/features_1_training_set_2.csv", converters={"word_tok": lambda x: x.strip("[]").replace("'","").split(", ")})
#training_data = pd.read_csv("csv_files/features_1_training_set_2_2.csv", converters={"word_tok": lambda x: x.strip("[]").replace("'","").split(", ")})
#training_data = pd.read_csv("csv_files/features_1_training_set_3.csv", converters={"word_tok": lambda x: x.strip("[]").replace("'","").split(", ")})
#training_data = pd.read_csv("csv_files/features_1_training_set_4.csv", converters={"word_tok": lambda x: x.strip("[]").replace("'","").split(", ")})
#training_data = pd.read_csv("csv_files/features_1_training_set_5.csv", converters={"word_tok": lambda x: x.strip("[]").replace("'","").split(", ")})
#training_data = pd.read_csv("csv_files/features_1_training_set_6.csv", converters={"word_tok": lambda x: x.strip("[]").replace("'","").split(", ")})
#training_data = pd.read_csv("csv_files/features_1_training_set_7.csv", converters={"word_tok": lambda x: x.strip("[]").replace("'","").split(", ")})
training_data = pd.read_csv("csv_files/features_1_training_set_8.csv", converters={"word_tok": lambda x: x.strip("[]").replace("'","").split(", ")})

In [779]:
training_data

,grade,normalised_docs,word_tok,sentence_tok,stemmed_word_token,stemmed_no_stopwords,nr_stopwords,avg_sentence_len_words,unique_words,word_count,...,PRON,PROPN,NOUN,nr_spelling_errors,nr_capitalization_errors,nr_punctuation_errors,grade_as_feature,avg_cosine_similariy_high_grade,pattern_cosine,weighted_cosine
0,34.0,a long time ago when i was in third grade i h...,"[A, long, time, ago, when, I, was, in, third, ...","['""A long time ago when I was in third grade I...","['a', 'long', 'time', 'ago', 'when', 'i', 'was...","['long', 'time', 'ago', 'third', 'grade', 'fri...",281,15.0,259,679,...,47,7,80,14,4,17,26.0,0.285749,0.000000,0.000000
1,46.0,softball has to be one of the single most gre...,"[Softball, has, to, be, one, of, the, single, ...",['Softball has to be one of the single most gr...,"['softbal', 'has', 'to', 'be', 'one', 'of', 't...","['softbal', 'one', 'singl', 'greatest', 'sport...",348,12.0,308,785,...,49,5,99,7,27,13,40.0,0.285749,0.000000,0.000000
2,40.0,"some people like making people laugh, i love ...","[Some, people, like, making, people, laugh, I,...","['Some people like making people laugh', 'I lo...","['some', 'peopl', 'like', 'make', 'peopl', 'la...","['peopl', 'like', 'make', 'peopl', 'laugh', 'l...",381,10.0,323,861,...,25,4,72,9,34,9,36.0,0.285749,6.908973,-6.908973
3,30.0,"""laughter"" @caps1 i hang out with my friends...","[LAUGHTER, CAPS1, I, hang, out, with, my, frie...","['""LAUGHTER"" @CAPS1 I hang out with my friend...","['laughter', 'caps1', 'i', 'hang', 'out', 'wit...","['laughter', 'caps1', 'hang', 'friend', 'one',...",378,15.0,255,712,...,32,38,132,5,24,4,40.0,0.285749,13.552038,-13.552038
4,26.0,well ima tell a story about the time i got @ca...,"[Well, i, m, a, tell, a, story, about, the, ti...",['Well ima tell a story about the time i got @...,"['well', 'i', 'm', 'a', 'tell', 'a', 'stori', ...","['well', 'tell', 'stori', 'time', 'got', 'caps...",370,11.0,203,671,...,41,3,108,28,19,3,34.0,0.285749,19.177102,-19.177102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,35.0,in most stories mothers and daughters are eit...,"[In, most, stories, mothers, and, daughters, a...",['In most stories mothers and daughters are ei...,"['in', 'most', 'stori', 'mother', 'and', 'daug...","['stori', 'mother', 'daughter', 'either', 'ene...",399,9.0,342,862,...,38,4,95,7,48,2,40.0,0.285749,1984.037703,1984.037703
719,32.0,i never understood the meaning laughter is th...,"[I, never, understood, the, meaning, laughter,...",['I never understood the meaning laughter is t...,"['i', 'never', 'understood', 'the', 'mean', 'l...","['never', 'understood', 'mean', 'laughter', 's...",259,9.0,218,556,...,79,7,121,11,25,4,40.0,0.285749,2128.729145,2128.729145
720,40.0,"when you laugh, is @caps5 out of habit, or is ...","[When, you, laugh, is, CAPS5, out, of, habit, ...","['When you laugh', 'is @CAPS5 out of habit', '...","['when', 'you', 'laugh', 'is', 'caps5', 'out',...","['laugh', 'caps5', 'habit', 'caps1', 'caus', '...",390,9.0,369,835,...,54,4,147,5,34,3,40.0,0.285749,11.891063,11.891063
721,40.0,trippin' on fen...,"[Trippin, on, fences, I, am, NUM1, years, youn...","['""Trippin on fences I am @NUM1 years young', ...","['trippin', 'on', 'fenc', 'i', 'am', 'num1', '...","['trippin', 'fenc', 'num1', 'year', 'young', '...",272,7.0,250,576,...,27,14,70,10,33,1,41.0,0.285749,23.782125,23.782125


In [780]:
training_data.isna().any().any()

False

# Split theses into multiple overlapping parts

In [781]:
each_theses_length = training_data["word_tok"].apply(len)
each_theses_length

0      679
1      785
2      861
3      712
4      671
      ... 
718    862
719    556
720    835
721    576
722    475
Name: word_tok, Length: 723, dtype: int64

In [782]:
avg_thesis_length = round(each_theses_length.mean())
avg_thesis_length

614

In [783]:
window_size = int(avg_thesis_length * 0.25)
window_size

153

In [784]:
training_data["theses_parts"] = training_data["word_tok"].apply(lambda word_list: [word_list[i:i + window_size] for i in range(0, len(word_list), 10)])
training_data["theses_parts"]

0      [[A, long, time, ago, when, I, was, in, third,...
1      [[Softball, has, to, be, one, of, the, single,...
2      [[Some, people, like, making, people, laugh, I...
3      [[LAUGHTER, CAPS1, I, hang, out, with, my, fri...
4      [[Well, i, m, a, tell, a, story, about, the, t...
                             ...                        
718    [[In, most, stories, mothers, and, daughters, ...
719    [[I, never, understood, the, meaning, laughter...
720    [[When, you, laugh, is, CAPS5, out, of, habit,...
721    [[Trippin, on, fences, I, am, NUM1, years, you...
722    [[Many, people, believe, that, laughter, can, ...
Name: theses_parts, Length: 723, dtype: object

In [785]:
training_data["corpus"] = training_data["theses_parts"].apply(lambda theses_part: [" ".join(word_list) for word_list in theses_part])
training_data["corpus"]

0      [A long time ago when I was in third grade I h...
1      [Softball has to be one of the single most gre...
2      [Some people like making people laugh I love i...
3      [LAUGHTER CAPS1 I hang out with my friends the...
4      [Well i m a tell a story about the time i got ...
                             ...                        
718    [In most stories mothers and daughters are eit...
719    [I never understood the meaning laughter is th...
720    [When you laugh is CAPS5 out of habit or is CA...
721    [Trippin on fences I am NUM1 years young and i...
722    [Many people believe that laughter can improve...
Name: corpus, Length: 723, dtype: object

In [786]:
# number of theses parts = nr of Tfidf vectors
# starts from 0 so -1
training_data["nr_thesis_parts"] = training_data["corpus"].apply(lambda text_part: len(text_part))
training_data["nr_thesis_parts"]

0      68
1      79
2      87
3      72
4      68
       ..
718    87
719    56
720    84
721    58
722    48
Name: nr_thesis_parts, Length: 723, dtype: int64

In [787]:
save_short_essays = training_data[training_data["nr_thesis_parts"] <= 1].copy()

# remove short essays for now
training_data = training_data[training_data["nr_thesis_parts"] > 1].copy()
training_data.head()

,grade,normalised_docs,word_tok,sentence_tok,stemmed_word_token,stemmed_no_stopwords,nr_stopwords,avg_sentence_len_words,unique_words,word_count,...,nr_spelling_errors,nr_capitalization_errors,nr_punctuation_errors,grade_as_feature,avg_cosine_similariy_high_grade,pattern_cosine,weighted_cosine,theses_parts,corpus,nr_thesis_parts
0,34.0,a long time ago when i was in third grade i h...,"[A, long, time, ago, when, I, was, in, third, ...","['""A long time ago when I was in third grade I...","['a', 'long', 'time', 'ago', 'when', 'i', 'was...","['long', 'time', 'ago', 'third', 'grade', 'fri...",281,15.0,259,679,...,14,4,17,26.0,0.285749,0.000000,0.000000,"[[A, long, time, ago, when, I, was, in, third,...",[A long time ago when I was in third grade I h...,68
1,46.0,softball has to be one of the single most gre...,"[Softball, has, to, be, one, of, the, single, ...",['Softball has to be one of the single most gr...,"['softbal', 'has', 'to', 'be', 'one', 'of', 't...","['softbal', 'one', 'singl', 'greatest', 'sport...",348,12.0,308,785,...,7,27,13,40.0,0.285749,0.000000,0.000000,"[[Softball, has, to, be, one, of, the, single,...",[Softball has to be one of the single most gre...,79
2,40.0,"some people like making people laugh, i love ...","[Some, people, like, making, people, laugh, I,...","['Some people like making people laugh', 'I lo...","['some', 'peopl', 'like', 'make', 'peopl', 'la...","['peopl', 'like', 'make', 'peopl', 'laugh', 'l...",381,10.0,323,861,...,9,34,9,36.0,0.285749,6.908973,-6.908973,"[[Some, people, like, making, people, laugh, I...",[Some people like making people laugh I love i...,87
3,30.0,"""laughter"" @caps1 i hang out with my friends...","[LAUGHTER, CAPS1, I, hang, out, with, my, frie...","['""LAUGHTER"" @CAPS1 I hang out with my friend...","['laughter', 'caps1', 'i', 'hang', 'out', 'wit...","['laughter', 'caps1', 'hang', 'friend', 'one',...",378,15.0,255,712,...,5,24,4,40.0,0.285749,13.552038,-13.552038,"[[LAUGHTER, CAPS1, I, hang, out, with, my, fri...",[LAUGHTER CAPS1 I hang out with my friends the...,72
4,26.0,well ima tell a story about the time i got @ca...,"[Well, i, m, a, tell, a, story, about, the, ti...",['Well ima tell a story about the time i got @...,"['well', 'i', 'm', 'a', 'tell', 'a', 'stori', ...","['well', 'tell', 'stori', 'time', 'got', 'caps...",370,11.0,203,671,...,28,19,3,34.0,0.285749,19.177102,-19.177102,"[[Well, i, m, a, tell, a, story, about, the, t...",[Well i m a tell a story about the time i got ...,68


# TFIDF vectors

In [788]:
# TFIDF
vectorizer = TfidfVectorizer()
training_data["TFIDF"] = training_data["corpus"].apply(vectorizer.fit_transform)
training_data["TFIDF"]

0        (0, 35)\t0.07492133308391932\n  (0, 139)\t0....
1        (0, 249)\t0.047173585111079296\n  (0, 151)\t...
2        (0, 18)\t0.05742009437708632\n  (0, 65)\t0.0...
3        (0, 1)\t0.036307368663450655\n  (0, 4)\t0.03...
4        (0, 26)\t0.03815729697580636\n  (0, 149)\t0....
                             ...                        
718      (0, 217)\t0.04753506906737664\n  (0, 110)\t0...
719      (0, 66)\t0.064224465238445\n  (0, 113)\t0.06...
720      (0, 332)\t0.02472833755589354\n  (0, 2)\t0.0...
721      (0, 225)\t0.06672803337634182\n  (0, 30)\t0....
722      (0, 100)\t0.04261248587729716\n  (0, 130)\t0...
Name: TFIDF, Length: 722, dtype: object

In [789]:
training_data["TFIDF"].iloc[0]

<68x243 sparse matrix of type '<class 'numpy.float64'>'
	with 5478 stored elements in Compressed Sparse Row format>

# Semantic coherence attributes (Euclidian distance & cosine similarity)

In [790]:
training_data["TFIDF_vector"] = training_data["TFIDF"].apply(lambda x: x.toarray())
training_data["TFIDF_vector"]

0      [[0.0, 0.1382133935717067, 0.1157765617931835,...
1      [[0.0, 0.08130017871482748, 0.0, 0.10544538355...
2      [[0.0, 0.0, 0.10192836995247587, 0.0, 0.0, 0.0...
3      [[0.08604198729883417, 0.036307368663450655, 0...
4      [[0.07357637214147816, 0.0, 0.0, 0.0, 0.116569...
                             ...                        
718    [[0.0, 0.0, 0.0, 0.08396974808670399, 0.086277...
719    [[0.04584147766752948, 0.0, 0.0372544085137157...
720    [[0.0, 0.06869252355430468, 0.0622888963321292...
721    [[0.08563719166391544, 0.09773176275129315, 0....
722    [[0.10474777998671708, 0.12302548158438405, 0....
Name: TFIDF_vector, Length: 722, dtype: object

### Distance Matrix

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html#scipy.spatial.distance.pdist

* cosine_distance = 1 - cosine_similarity
* cosine_similarity = 1 - cosine_distance

In [791]:
def get_distance_matrix(sparse_m, dist_metric, full_matrix = True):

    X = sparse_m.todense()
    dist_m = scipy.spatial.distance.pdist(X, metric = dist_metric)

    if full_matrix:
        # form condensed matrix to full matrix
        full_m = scipy.spatial.distance.squareform(dist_m)
        return full_m
    else:
        return dist_m

In [792]:
training_data["euclidian_distance_matrix"] = training_data["TFIDF"].apply(lambda m: get_distance_matrix(m, dist_metric = "euclidean"))
training_data["euclidian_distance_matrix"].iloc[0]

array([[0.        , 0.34344266, 0.44826083, ..., 1.33008588, 1.37409994,
        1.38599382],
       [0.34344266, 0.        , 0.2725315 , ..., 1.33098428, 1.37345615,
        1.39004629],
       [0.44826083, 0.2725315 , 0.        , ..., 1.31718043, 1.36722937,
        1.3946548 ],
       ...,
       [1.33008588, 1.33098428, 1.31718043, ..., 0.        , 0.48325022,
        0.84931995],
       [1.37409994, 1.37345615, 1.36722937, ..., 0.48325022, 0.        ,
        0.78902671],
       [1.38599382, 1.39004629, 1.3946548 , ..., 0.84931995, 0.78902671,
        0.        ]])

In [793]:
# cosine distance not cosine similarity
training_data["cosine_distance_matrix"] = training_data["TFIDF"].apply(lambda m: get_distance_matrix(m, dist_metric = "cosine"))
training_data["cosine_distance_matrix"]

0      [[0.0, 0.05897643091158067, 0.1004688838429044...
1      [[0.0, 0.04414812978987592, 0.0944659382057905...
2      [[0.0, 0.03281673965419385, 0.0778123965351562...
3      [[0.0, 0.047924349030667446, 0.085782170356536...
4      [[0.0, 0.04099141916654414, 0.0691034861564843...
                             ...                        
718    [[0.0, 0.0789749013140616, 0.14460113138063324...
719    [[0.0, 0.0501778783417256, 0.07890967154182638...
720    [[0.0, 0.032916360663586275, 0.074313822430500...
721    [[0.0, 0.06592245216095416, 0.1590587733799970...
722    [[0.0, 0.05118834582332499, 0.1271396151684704...
Name: cosine_distance_matrix, Length: 722, dtype: object

In [794]:
training_data["cosine_similarity_matrix"] = training_data["cosine_distance_matrix"].apply(lambda matrix: [1 - dist for dist in matrix])
training_data["cosine_similarity_matrix"]

0      [[1.0, 0.9410235690884193, 0.8995311161570956,...
1      [[1.0, 0.9558518702101241, 0.9055340617942095,...
2      [[1.0, 0.9671832603458062, 0.9221876034648437,...
3      [[1.0, 0.9520756509693326, 0.9142178296434633,...
4      [[1.0, 0.9590085808334559, 0.9308965138435157,...
                             ...                        
718    [[1.0, 0.9210250986859384, 0.8553988686193668,...
719    [[1.0, 0.9498221216582744, 0.9210903284581736,...
720    [[1.0, 0.9670836393364137, 0.9256861775694994,...
721    [[1.0, 0.9340775478390458, 0.8409412266200029,...
722    [[1.0, 0.948811654176675, 0.8728603848315296, ...
Name: cosine_similarity_matrix, Length: 722, dtype: object

In [795]:
# get distance to neighbouring vector

training_data["eucl_dist_neighbour"] = training_data.apply(lambda x: [val[i + 1:][0] for i, val in enumerate(x.euclidian_distance_matrix) if i < len(x.euclidian_distance_matrix) - 1], axis = 1)
training_data["eucl_dist_neighbour"]

0      [0.3434426616236857, 0.27253150065656184, 0.38...
1      [0.29714686533724843, 0.27666235586270216, 0.2...
2      [0.25619031852977464, 0.2882261379041853, 0.35...
3      [0.3095944089632991, 0.28441145170284127, 0.28...
4      [0.28632645412725743, 0.2669284709157069, 0.30...
                             ...                        
718    [0.39742899067396, 0.2928953125921563, 0.33423...
719    [0.3167897673275623, 0.23868265076479858, 0.31...
720    [0.25657887934740986, 0.2618133472486362, 0.24...
721    [0.36310453635545337, 0.3509454420598844, 0.26...
722    [0.3199635786252082, 0.37657651575169954, 0.33...
Name: eucl_dist_neighbour, Length: 722, dtype: object

In [796]:
# get distance to neighbouring vector

training_data["cos_dist_neighbour"] = training_data.apply(lambda x: [val[i + 1:][0] for i, val in enumerate(x.euclidian_distance_matrix) if i < len(x.euclidian_distance_matrix) - 1], axis = 1)
training_data["cos_dist_neighbour"]

0      [0.3434426616236857, 0.27253150065656184, 0.38...
1      [0.29714686533724843, 0.27666235586270216, 0.2...
2      [0.25619031852977464, 0.2882261379041853, 0.35...
3      [0.3095944089632991, 0.28441145170284127, 0.28...
4      [0.28632645412725743, 0.2669284709157069, 0.30...
                             ...                        
718    [0.39742899067396, 0.2928953125921563, 0.33423...
719    [0.3167897673275623, 0.23868265076479858, 0.31...
720    [0.25657887934740986, 0.2618133472486362, 0.24...
721    [0.36310453635545337, 0.3509454420598844, 0.26...
722    [0.3199635786252082, 0.37657651575169954, 0.33...
Name: cos_dist_neighbour, Length: 722, dtype: object

In [797]:
training_data["cos_similarity_neighbour"] = training_data["cos_dist_neighbour"].apply(lambda cos_distances: [1 - cos_dist for cos_dist in cos_distances])
training_data["cos_similarity_neighbour"]

0      [0.6565573383763144, 0.7274684993434382, 0.614...
1      [0.7028531346627516, 0.7233376441372978, 0.723...
2      [0.7438096814702253, 0.7117738620958147, 0.642...
3      [0.6904055910367008, 0.7155885482971587, 0.711...
4      [0.7136735458727426, 0.7330715290842931, 0.696...
                             ...                        
718    [0.60257100932604, 0.7071046874078437, 0.66576...
719    [0.6832102326724376, 0.7613173492352014, 0.683...
720    [0.7434211206525901, 0.7381866527513639, 0.757...
721    [0.6368954636445466, 0.6490545579401156, 0.734...
722    [0.6800364213747918, 0.6234234842483004, 0.665...
Name: cos_similarity_neighbour, Length: 722, dtype: object

In [798]:
# get distance to closest neighbouring vectors (nearest neighbour)

training_data["eucl_dist_nn"] = training_data.apply(lambda x: [(np.sort(val[i+1:]))[0] for i, val in enumerate(x.euclidian_distance_matrix) if i < len(x.euclidian_distance_matrix) - 1], axis = 1)
training_data["eucl_dist_nn"]

0      [0.3434426616236857, 0.27253150065656184, 0.38...
1      [0.29714686533724843, 0.27666235586270216, 0.2...
2      [0.25619031852977464, 0.2882261379041853, 0.35...
3      [0.3095944089632991, 0.28441145170284127, 0.28...
4      [0.28632645412725743, 0.2669284709157069, 0.30...
                             ...                        
718    [0.39742899067396, 0.2928953125921563, 0.33423...
719    [0.3167897673275623, 0.23868265076479858, 0.31...
720    [0.25657887934740986, 0.2618133472486362, 0.24...
721    [0.36310453635545337, 0.3509454420598844, 0.26...
722    [0.3199635786252082, 0.37657651575169954, 0.33...
Name: eucl_dist_nn, Length: 722, dtype: object

In [799]:
# get the distance to the neighbour
training_data["cos_dist_nn"] = training_data.apply(lambda x: [(np.sort(val[i+1:]))[0] for i, val in enumerate(x.cosine_distance_matrix) if i < len(x.cosine_distance_matrix) - 1], axis = 1)
training_data["cos_dist_nn"]

0      [0.05897643091158067, 0.03713670942505898, 0.0...
1      [0.04414812978987592, 0.03827102957575024, 0.0...
2      [0.03281673965419385, 0.04153715328558116, 0.0...
3      [0.047924349030667446, 0.0404449369298594, 0.0...
4      [0.04099141916654414, 0.035625404292699336, 0....
                             ...                        
718    [0.0789749013140616, 0.042893832069228255, 0.0...
719    [0.0501778783417256, 0.028484703888055685, 0.0...
720    [0.032916360663586275, 0.03427311439876746, 0....
721    [0.06592245216095416, 0.06158135165130407, 0.0...
722    [0.05118834582332499, 0.07090493610784465, 0.0...
Name: cos_dist_nn, Length: 722, dtype: object

In [800]:
# get the distance to the neighbour
training_data["cos_similarity_nn"] = training_data["cos_dist_nn"].apply(lambda cos_distances: [1 - cos_dist for cos_dist in cos_distances])
training_data["cos_similarity_nn"]

0      [0.9410235690884193, 0.962863290574941, 0.9257...
1      [0.9558518702101241, 0.9617289704242498, 0.961...
2      [0.9671832603458062, 0.9584628467144188, 0.936...
3      [0.9520756509693326, 0.9595550630701406, 0.958...
4      [0.9590085808334559, 0.9643745957073007, 0.954...
                             ...                        
718    [0.9210250986859384, 0.9571061679307717, 0.944...
719    [0.9498221216582744, 0.9715152961119443, 0.949...
720    [0.9670836393364137, 0.9657268856012325, 0.970...
721    [0.9340775478390458, 0.9384186483486959, 0.964...
722    [0.948811654176675, 0.9290950638921553, 0.9441...
Name: cos_similarity_nn, Length: 722, dtype: object

### All Distances

In [801]:
training_data["euclidian_distances"] = training_data["TFIDF"].apply(lambda m: get_distance_matrix(m, dist_metric = "euclidean", full_matrix = False))
training_data["euclidian_distances"]

0      [0.3434426616236857, 0.4482608255087754, 0.600...
1      [0.29714686533724843, 0.4346629457540416, 0.51...
2      [0.25619031852977464, 0.39449308367867847, 0.5...
3      [0.3095944089632991, 0.41420326014297976, 0.51...
4      [0.28632645412725743, 0.3717619834154222, 0.50...
                             ...                        
718    [0.39742899067396, 0.5377752902107591, 0.63915...
719    [0.3167897673275623, 0.39726482739307795, 0.50...
720    [0.25657887934740986, 0.385522560767849, 0.471...
721    [0.36310453635545337, 0.5640191014141223, 0.63...
722    [0.3199635786252082, 0.5042610735888122, 0.605...
Name: euclidian_distances, Length: 722, dtype: object

In [802]:
training_data["cosine_distances"] = training_data["TFIDF"].apply(lambda m: get_distance_matrix(m, dist_metric = "cosine", full_matrix = False))
training_data["cosine_distances"]

0      [0.05897643091158067, 0.10046888384290442, 0.1...
1      [0.04414812978987592, 0.09446593820579052, 0.1...
2      [0.03281673965419385, 0.07781239653515626, 0.1...
3      [0.047924349030667446, 0.08578217035653668, 0....
4      [0.04099141916654414, 0.06910348615648432, 0.1...
                             ...                        
718    [0.0789749013140616, 0.14460113138063324, 0.20...
719    [0.0501778783417256, 0.07890967154182638, 0.12...
720    [0.032916360663586275, 0.07431382243050055, 0....
721    [0.06592245216095416, 0.15905877337999708, 0.2...
722    [0.05118834582332499, 0.1271396151684704, 0.18...
Name: cosine_distances, Length: 722, dtype: object

In [803]:
training_data["cosine_similarities"] = training_data["cosine_distances"].apply(lambda cos_dist: 1 - cos_dist)
training_data["cosine_similarities"]

0      [0.9410235690884193, 0.8995311161570956, 0.819...
1      [0.9558518702101241, 0.9055340617942095, 0.866...
2      [0.9671832603458062, 0.9221876034648437, 0.846...
3      [0.9520756509693326, 0.9142178296434633, 0.869...
4      [0.9590085808334559, 0.9308965138435157, 0.874...
                             ...                        
718    [0.9210250986859384, 0.8553988686193668, 0.795...
719    [0.9498221216582744, 0.9210903284581736, 0.872...
720    [0.9670836393364137, 0.9256861775694994, 0.888...
721    [0.9340775478390458, 0.8409412266200029, 0.796...
722    [0.948811654176675, 0.8728603848315296, 0.8168...
Name: cosine_similarities, Length: 722, dtype: object

# Basic Coherence Measures


 Change coseine distance to cosine similarity

* Cosine Distance = 1 - Cosine Similarity
* Cosine Similarity = 1 - Cosine Distance

## Average distance between any two points (2x)

In [804]:
training_data["avg_eucl_dist"] = training_data["euclidian_distances"].apply(np.mean)
training_data["avg_eucl_dist"]

0      1.023180
1      1.067530
2      1.106320
3      1.026832
4      0.972721
         ...   
718    1.062556
719    1.013773
720    1.092305
721    1.031751
722    1.000312
Name: avg_eucl_dist, Length: 722, dtype: float64

In [805]:
training_data["avg_cos_dist"] = training_data["cosine_distances"].apply(np.mean)
training_data["avg_cos_dist"]

0      0.560944
1      0.604392
2      0.648943
3      0.562906
4      0.507182
         ...   
718    0.595961
719    0.561965
720    0.628564
721    0.578545
722    0.546817
Name: avg_cos_dist, Length: 722, dtype: float64

In [806]:
training_data["avg_cos_similarity"] = training_data["cosine_similarities"].apply(np.mean)
training_data["avg_cos_similarity"]

0      0.439056
1      0.395608
2      0.351057
3      0.437094
4      0.492818
         ...   
718    0.404039
719    0.438035
720    0.371436
721    0.421455
722    0.453183
Name: avg_cos_similarity, Length: 722, dtype: float64

# Maximum difference(distance) between any two points

In [807]:
training_data["max_eucl_dist"] = training_data["euclidian_distances"].apply(np.max)
training_data["max_eucl_dist"]

0      1.406485
1      1.388623
2      1.414214
3      1.396542
4      1.414214
         ...   
718    1.414214
719    1.414214
720    1.414214
721    1.375512
722    1.414214
Name: max_eucl_dist, Length: 722, dtype: float64

In [808]:
training_data["max_cos_similarity"] = training_data["cosine_similarities"].apply(np.max)
training_data["max_cos_similarity"]

0      0.986793
1      0.977745
2      0.987549
3      0.980955
4      0.983596
         ...   
718    0.984662
719    0.984826
720    0.973800
721    0.984202
722    0.981622
Name: max_cos_similarity, Length: 722, dtype: float64

## Minimum Distance between neighbouring points (2x)

In [809]:
training_data["min_neighbouring_eucl_dist"] = training_data["eucl_dist_neighbour"].apply(lambda dist: np.array(dist).min())
training_data["min_neighbouring_eucl_dist"]

0      0.162524
1      0.210972
2      0.157805
3      0.195168
4      0.181132
         ...   
718    0.175146
719    0.174205
720    0.228913
721    0.177750
722    0.191719
Name: min_neighbouring_eucl_dist, Length: 722, dtype: float64

In [810]:
training_data["min_neighbouring_cos_dist"] = training_data["cos_dist_neighbour"].apply(lambda dist: np.array(dist).min())
training_data["min_neighbouring_cos_dist"]

0      0.162524
1      0.210972
2      0.157805
3      0.195168
4      0.181132
         ...   
718    0.175146
719    0.174205
720    0.228913
721    0.177750
722    0.191719
Name: min_neighbouring_cos_dist, Length: 722, dtype: float64

In [811]:
training_data["min_neighbouring_cos_similarity"] = training_data["cos_similarity_neighbour"].apply(lambda dist: np.array(dist).min())
training_data["min_neighbouring_cos_similarity"]

0      0.210973
1      0.034945
2     -0.326379
3     -0.096903
4     -0.131577
         ...   
718    0.014270
719    0.080145
720    0.103982
721    0.108835
722    0.247931
Name: min_neighbouring_cos_similarity, Length: 722, dtype: float64

## Maximum Distance between neighbouring points (2x) 

In [812]:
training_data["max_neighbouring_eucl_dist"] = training_data["eucl_dist_neighbour"].apply(lambda dist: np.array(dist).max())
training_data["max_neighbouring_eucl_dist"]

0      0.789027
1      0.965055
2      1.326379
3      1.096903
4      1.131577
         ...   
718    0.985730
719    0.919855
720    0.896018
721    0.891165
722    0.752069
Name: max_neighbouring_eucl_dist, Length: 722, dtype: float64

In [813]:
training_data["max_neighbouring_cos_dist"] = training_data["cos_dist_neighbour"].apply(lambda dist: np.array(dist).max())
training_data["max_neighbouring_cos_dist"]

0      0.789027
1      0.965055
2      1.326379
3      1.096903
4      1.131577
         ...   
718    0.985730
719    0.919855
720    0.896018
721    0.891165
722    0.752069
Name: max_neighbouring_cos_dist, Length: 722, dtype: float64

In [814]:
training_data["max_neighbouring_cos_similarity"] = training_data["cos_similarity_neighbour"].apply(lambda dist: np.array(dist).max())
training_data["max_neighbouring_cos_similarity"]

0      0.837476
1      0.789028
2      0.842195
3      0.804832
4      0.818868
         ...   
718    0.824854
719    0.825795
720    0.771087
721    0.822250
722    0.808281
Name: max_neighbouring_cos_similarity, Length: 722, dtype: float64

## Quotient of Min/Max Distance between neighbouring points (2x) 

In [815]:
training_data["eucl_min_max_quotient"] = training_data.apply(lambda x: x.min_neighbouring_eucl_dist / x.max_neighbouring_eucl_dist if abs(x.max_neighbouring_eucl_dist) > 0 else 0, axis = 1)
training_data["eucl_min_max_quotient"]

0      0.205981
1      0.218611
2      0.118974
3      0.177926
4      0.160070
         ...   
718    0.177681
719    0.189383
720    0.255478
721    0.199458
722    0.254922
Name: eucl_min_max_quotient, Length: 722, dtype: float64

In [816]:
training_data["cos_min_max_quotient"] = training_data.apply(lambda x: x.min_neighbouring_cos_similarity / x.max_neighbouring_cos_similarity if abs(x.max_neighbouring_cos_similarity) > 0 else 0, axis = 1)
training_data["cos_min_max_quotient"]

0      0.251916
1      0.044288
2     -0.387534
3     -0.120401
4     -0.160681
         ...   
718    0.017300
719    0.097052
720    0.134851
721    0.132363
722    0.306738
Name: cos_min_max_quotient, Length: 722, dtype: float64

## Average distance between neighboring points (2x)

In [817]:
training_data["avg_neighbouring_eucl_dist"] = training_data["eucl_dist_neighbour"].apply(lambda dist: np.array(dist).mean())
training_data["avg_neighbouring_eucl_dist"]

0      0.286399
1      0.296551
2      0.284852
3      0.293724
4      0.284645
         ...   
718    0.298230
719    0.280517
720    0.308581
721    0.292770
722    0.316295
Name: avg_neighbouring_eucl_dist, Length: 722, dtype: float64

In [818]:
training_data["avg_neighbouring_cos_dist"] = training_data["cos_dist_neighbour"].apply(lambda dist: np.array(dist).mean())
training_data["avg_neighbouring_cos_dist"]

0      0.286399
1      0.296551
2      0.284852
3      0.293724
4      0.284645
         ...   
718    0.298230
719    0.280517
720    0.308581
721    0.292770
722    0.316295
Name: avg_neighbouring_cos_dist, Length: 722, dtype: float64

In [819]:
training_data["avg_neighbouring_cos_similarity"] = training_data["cos_similarity_neighbour"].apply(lambda dist: np.array(dist).mean())
training_data["avg_neighbouring_cos_similarity"]

0      0.713601
1      0.703449
2      0.715148
3      0.706276
4      0.715355
         ...   
718    0.701770
719    0.719483
720    0.691419
721    0.707230
722    0.683705
Name: avg_neighbouring_cos_similarity, Length: 722, dtype: float64

## Clark and Evans’ distance to the nearest neighbor ***

In [820]:
training_data["eucl_clark_evan_dist_nn"] = training_data.apply(lambda x: (2 * np.sqrt(len(x.eucl_dist_nn)) * sum(x.eucl_dist_nn)) / len(x.eucl_dist_nn), axis = 1)
training_data["eucl_clark_evan_dist_nn"]

0      4.688562
1      5.238134
2      5.283220
3      4.949919
4      4.659844
         ...   
718    5.531345
719    4.160739
720    5.622612
721    4.420725
722    4.336825
Name: eucl_clark_evan_dist_nn, Length: 722, dtype: float64

In [821]:
training_data["cos_clark_evan_dist_nn"] = training_data.apply(lambda x: (2 * np.sqrt(len(x.cos_similarity_nn)) * sum(x.cos_similarity_nn)) / len(x.cos_similarity_nn), axis = 1)
training_data["cos_clark_evan_dist_nn"]

0      15.638643
1      16.809977
2      17.659164
3      16.000071
4      15.550602
         ...    
718    17.630718
719    14.162076
720    17.289607
721    14.349812
722    12.937942
Name: cos_clark_evan_dist_nn, Length: 722, dtype: float64

## Average distance to the neirest neighbour

In [822]:
training_data["avg_eucl_dist_nn"] = training_data["eucl_dist_nn"].apply(np.mean)
training_data["avg_eucl_dist_nn"]

0      0.286399
1      0.296551
2      0.284852
3      0.293724
4      0.284645
         ...   
718    0.298230
719    0.280517
720    0.308581
721    0.292770
722    0.316295
Name: avg_eucl_dist_nn, Length: 722, dtype: float64

In [823]:
training_data["avg_cos_dist_nn"] = training_data["cos_dist_nn"].apply(np.mean)
training_data["avg_cos_dist_nn"]

0      0.044718
1      0.048322
2      0.047882
3      0.050570
4      0.050096
         ...   
718    0.049415
719    0.045193
720    0.051110
721    0.049660
722    0.056404
Name: avg_cos_dist_nn, Length: 722, dtype: float64

In [824]:
training_data["avg_cos_similarity_nn"] = training_data["cos_similarity_nn"].apply(np.mean)
training_data["avg_cos_similarity_nn"]

0      0.955282
1      0.951678
2      0.952118
3      0.949430
4      0.949904
         ...   
718    0.950585
719    0.954807
720    0.948890
721    0.950340
722    0.943596
Name: avg_cos_similarity_nn, Length: 722, dtype: float64

## Cumulative frequency distribution G of the nearest neighbors’ distances

In [825]:
training_data["eucl_cumulative_freq_distribution"] = training_data.apply(lambda x: abs(np.array(x.eucl_dist_nn)[np.array(x.eucl_dist_nn) <= x.avg_eucl_dist_nn])/len(x.eucl_dist_nn), axis = 1)
training_data["eucl_cumulative_freq_distribution"]

0      [0.00406763433815764, 0.0037056903240001936, 0...
1      [0.003546953280291053, 0.0035462155929452408, ...
2      [0.002978957192206682, 0.0030769921319905988, ...
3      [0.004005795094406215, 0.004060004438232743, 0...
4      [0.00398400702859264, 0.0035633380783663357, 0...
                             ...                        
718    [0.0034057594487460034, 0.0031361005162917413,...
719    [0.004339684559359974, 0.00443649511462985, 0....
720    [0.0030913117993663836, 0.003154377677694412, ...
721    [0.004653917840253136, 0.004886915250331405, 0...
722    [0.005811828671732947, 0.006081026708776762, 0...
Name: eucl_cumulative_freq_distribution, Length: 722, dtype: object

In [826]:
training_data["avg_eucl_cumulative_freq_distribution"] = training_data["eucl_cumulative_freq_distribution"].apply(lambda vec: np.array(vec).mean())
training_data["avg_eucl_cumulative_freq_distribution"]

0      0.003713
1      0.003321
2      0.002853
3      0.003625
4      0.003573
         ...   
718    0.003096
719    0.004297
720    0.003377
721    0.004319
722    0.005731
Name: avg_eucl_cumulative_freq_distribution, Length: 722, dtype: float64

In [827]:
training_data["cos_cumulative_freq_distribution"] = training_data.apply(lambda x: abs(np.array(x.cos_similarity_nn)[np.array(x.cos_similarity_nn) <= x.avg_cos_similarity_nn])/len(x.cos_similarity_nn), axis = 1)
training_data["cos_cumulative_freq_distribution"]

0      [0.014045127896842079, 0.013816788481681917, 0...
1      [0.012127624274223725, 0.012010067740714423, 0...
2      [0.010886749511260691, 0.010906907135324112, 0...
3      [0.013089800388392964, 0.013312109577398498, 0...
4      [0.014176579590618294, 0.013964201722340412, 0...
                             ...                        
718    [0.010709594170766725, 0.010978427843319934, 0...
719    [0.017269493121059534, 0.017269808581060905, 0...
720    [0.011408057431436571, 0.011393187294752949, 0...
721    [0.016387325400685014, 0.01646348505874905, 0....
722    [0.019767980082811817, 0.0200380024186599, 0.0...
Name: cos_cumulative_freq_distribution, Length: 722, dtype: object

In [828]:
training_data["avg_cos_cumulative_freq_distribution"] = training_data["cos_cumulative_freq_distribution"].apply(lambda vec: np.array(vec).mean())
training_data["avg_cos_cumulative_freq_distribution"]

0      0.013720
1      0.011773
2      0.010453
3      0.011638
4      0.012344
         ...   
718    0.010378
719    0.016220
720    0.010889
721    0.015313
722    0.018016
Name: avg_cos_cumulative_freq_distribution, Length: 722, dtype: float64

# Spatial Data Analysis

In [829]:
# calculate the centroid of all points
training_data["TFIDF_centroid"] = training_data["TFIDF"].apply(lambda x: np.average(x.toarray(), axis = 0))
training_data["TFIDF_centroid"]

0      [0.025078351529480847, 0.002032549905466275, 0...
1      [0.0401228825064311, 0.008198136821014364, 0.0...
2      [0.05213391609745086, 0.018474419469081276, 0....
3      [0.08549178695172073, 0.052649652433352095, 0....
4      [0.03545963019016269, 0.043749123898988634, 0....
                             ...                        
718    [0.015963207075289237, 0.02635589173433269, 0....
719    [0.07624400762000917, 0.04740629263556807, 0.0...
720    [0.01647511655510294, 0.012253711857395341, 0....
721    [0.013124600249852321, 0.04877728984995034, 0....
722    [0.006903666584652725, 0.0482229519999813, 0.0...
Name: TFIDF_centroid, Length: 722, dtype: object

In [830]:
# adjusted shape before adding centroid array to the other arrays
training_data["added_centroid"] = training_data.apply(lambda x: np.concatenate([x.TFIDF_centroid.reshape(1, -1), x.TFIDF.todense()]), axis = 1)
training_data["added_centroid"]

0      [[[[[0.02507835 0.00203255 0.03033583 0.032682...
1      [[[[[0.04012288 0.00819814 0.01700845 0.002727...
2      [[[[[0.05213392 0.01847442 0.01095669 0.046866...
3      [[[[[0.08549179 0.05264965 0.01921853 0.007860...
4      [[[[[0.03545963 0.04374912 0.02099514 0.043086...
                             ...                        
718    [[[[[0.01596321 0.02635589 0.01572556 0.012890...
719    [[[[[0.07624401 0.04740629 0.05991404 0.017914...
720    [[[[[0.01647512 0.01225371 0.01546376 0.015947...
721    [[[[[0.0131246  0.04877729 0.01889175 0.036904...
722    [[[[[0.00690367 0.04822295 0.01792584 0.037986...
Name: added_centroid, Length: 722, dtype: object

In [831]:
def get_centroid_distance_matrix(matrix, dist_metric, full_matrix = True):

    dist_m = scipy.spatial.distance.pdist(matrix, metric = dist_metric)

    if full_matrix:
        # form condensed matrix to full matrix
        full_m = scipy.spatial.distance.squareform(dist_m)
        return full_m
    else:
        return dist_m

In [832]:
training_data["eucl_centroid_distance_matrix"] = training_data.apply(lambda m: get_centroid_distance_matrix(m.added_centroid , dist_metric = "euclidean"), axis = 1)
training_data["eucl_centroid_distance_matrix"].iloc[0]

array([[0.        , 0.87443284, 0.84616064, ..., 0.95520049, 1.02385596,
        1.08388449],
       [0.87443284, 0.        , 0.34344266, ..., 1.33008588, 1.37409994,
        1.38599382],
       [0.84616064, 0.34344266, 0.        , ..., 1.33098428, 1.37345615,
        1.39004629],
       ...,
       [0.95520049, 1.33008588, 1.33098428, ..., 0.        , 0.48325022,
        0.84931995],
       [1.02385596, 1.37409994, 1.37345615, ..., 0.48325022, 0.        ,
        0.78902671],
       [1.08388449, 1.38599382, 1.39004629, ..., 0.84931995, 0.78902671,
        0.        ]])

In [833]:
training_data["cos_centroid_distance_matrix"] = training_data.apply(lambda m: get_centroid_distance_matrix(m.added_centroid , dist_metric = "cosine"), axis = 1)
training_data["cos_centroid_distance_matrix"].iloc[0]

array([[0.        , 0.48963557, 0.4532687 , ..., 0.60011195, 0.70169034,
        0.79627981],
       [0.48963557, 0.        , 0.05897643, ..., 0.88456422, 0.94407532,
        0.96048944],
       [0.4532687 , 0.05897643, 0.        , ..., 0.88575957, 0.94319089,
        0.96611434],
       ...,
       [0.60011195, 0.88456422, 0.88575957, ..., 0.        , 0.11676539,
        0.36067219],
       [0.70169034, 0.94407532, 0.94319089, ..., 0.11676539, 0.        ,
        0.31128157],
       [0.79627981, 0.96048944, 0.96611434, ..., 0.36067219, 0.31128157,
        0.        ]])

In [834]:
training_data["cos_centroid_similarity_matrix"] = training_data.apply(lambda m: 1 - get_centroid_distance_matrix(m.added_centroid , dist_metric = "cosine"), axis = 1)
training_data["cos_centroid_similarity_matrix"].iloc[0]

array([[1.        , 0.51036443, 0.5467313 , ..., 0.39988805, 0.29830966,
        0.20372019],
       [0.51036443, 1.        , 0.94102357, ..., 0.11543578, 0.05592468,
        0.03951056],
       [0.5467313 , 0.94102357, 1.        , ..., 0.11424043, 0.05680911,
        0.03388566],
       ...,
       [0.39988805, 0.11543578, 0.11424043, ..., 1.        , 0.88323461,
        0.63932781],
       [0.29830966, 0.05592468, 0.05680911, ..., 0.88323461, 1.        ,
        0.68871843],
       [0.20372019, 0.03951056, 0.03388566, ..., 0.63932781, 0.68871843,
        1.        ]])

### Distance of centroid with each point 
* (first row of distance matrix without 0 element)

In [835]:
training_data["eucl_centroid_distances"] = training_data["eucl_centroid_distance_matrix"].apply(lambda centroid_dist_m: centroid_dist_m[0][1:])
training_data["eucl_centroid_distances"]

0      [0.8744328423128997, 0.8461606410286182, 0.837...
1      [0.9284150120318378, 0.9038625869943485, 0.882...
2      [0.8693996964907765, 0.8701808936849607, 0.845...
3      [0.8594747727056868, 0.8380796043602232, 0.829...
4      [0.8020141125211879, 0.7983585269821908, 0.766...
                             ...                        
718    [0.8446311833648924, 0.8026627198939663, 0.800...
719    [0.8382347948753048, 0.830712008775461, 0.8162...
720    [0.9584461418822402, 0.9404669288621743, 0.917...
721    [0.8833904800500046, 0.8511294380385445, 0.823...
722    [0.9058450520588263, 0.8806061272442876, 0.847...
Name: eucl_centroid_distances, Length: 722, dtype: object

In [836]:
training_data["cos_centroid_distances"] = training_data["cos_centroid_distance_matrix"].apply(lambda centroid_dist_m: centroid_dist_m[0][1:])
training_data["cos_centroid_distances"]

0      [0.48963556648921513, 0.4532687022859717, 0.44...
1      [0.5737940247871554, 0.5383727740122273, 0.507...
2      [0.4967448050812774, 0.49787960872692705, 0.46...
3      [0.47061708799048274, 0.4433918001479239, 0.43...
4      [0.3941417701908353, 0.39000612560655645, 0.35...
                             ...                        
718    [0.4559434798360752, 0.4020170572179659, 0.399...
719    [0.4431921680934142, 0.4338139924047283, 0.416...
720    [0.6261151877697575, 0.5983837421571712, 0.563...
721    [0.5044014854939042, 0.46180511636560584, 0.42...
722    [0.527565764270915, 0.4944904580335623, 0.4519...
Name: cos_centroid_distances, Length: 722, dtype: object

In [837]:
training_data["cos_centroid_similarities"] = training_data["cos_centroid_similarity_matrix"].apply(lambda centroid_dist_m: centroid_dist_m[0][1:])
training_data["cos_centroid_similarities"]

0      [0.5103644335107849, 0.5467312977140283, 0.557...
1      [0.42620597521284465, 0.4616272259877727, 0.49...
2      [0.5032551949187226, 0.502120391273073, 0.5371...
3      [0.5293829120095173, 0.5566081998520761, 0.566...
4      [0.6058582298091647, 0.6099938743934435, 0.645...
                             ...                        
718    [0.5440565201639248, 0.5979829427820341, 0.600...
719    [0.5568078319065858, 0.5661860075952717, 0.583...
720    [0.3738848122302425, 0.40161625784282884, 0.43...
721    [0.4955985145060958, 0.5381948836343942, 0.573...
722    [0.47243423572908505, 0.5055095419664377, 0.54...
Name: cos_centroid_similarities, Length: 722, dtype: object

## Average distance between the centroid and each point (2x)

In [838]:
training_data["avg_eucl_centroid_distances"] = training_data["eucl_centroid_distances"].apply(np.mean)
training_data["avg_eucl_centroid_distances"]

0      0.738316
1      0.769604
2      0.798330
3      0.739620
4      0.701244
         ...   
718    0.764402
719    0.736932
720    0.785634
721    0.751063
722    0.724932
Name: avg_eucl_centroid_distances, Length: 722, dtype: float64

In [839]:
training_data["avg_cos_centroid_distances"] = training_data["cos_centroid_distances"].apply(np.mean)
training_data["avg_cos_centroid_distances"]

0      0.331191
1      0.364974
2      0.401238
3      0.332983
4      0.292698
         ...   
718    0.358993
719    0.330620
720    0.384436
721    0.343167
722    0.318402
Name: avg_cos_centroid_distances, Length: 722, dtype: float64

In [840]:
training_data["avg_cos_centroid_similarities"] = training_data["cos_centroid_similarities"].apply(lambda x: np.mean(x))
training_data["avg_cos_centroid_similarities"]

0      0.668809
1      0.635026
2      0.598762
3      0.667017
4      0.707302
         ...   
718    0.641007
719    0.669380
720    0.615564
721    0.656833
722    0.681598
Name: avg_cos_centroid_similarities, Length: 722, dtype: float64

## Minimal distance between the centroid and each point (2x)

In [841]:
training_data["min_eucl_centroid_distances"] = training_data["eucl_centroid_distances"].apply(np.min)
training_data["min_eucl_centroid_distances"]

0      0.627981
1      0.690605
2      0.719405
3      0.633794
4      0.612015
         ...   
718    0.693431
719    0.633374
720    0.710322
721    0.689771
722    0.609169
Name: min_eucl_centroid_distances, Length: 722, dtype: float64

In [842]:
training_data["min_cos_centroid_distances"] = training_data["cos_centroid_distances"].apply(np.min)
training_data["min_cos_centroid_distances"]

0      0.212820
1      0.270642
2      0.297741
3      0.217998
4      0.204220
         ...   
718    0.274545
719    0.218003
720    0.289789
721    0.272535
722    0.197849
Name: min_cos_centroid_distances, Length: 722, dtype: float64

In [843]:
training_data["min_cos_centroid_similarities"] = training_data["cos_centroid_similarities"].apply(np.min)
training_data["min_cos_centroid_similarities"]

0      0.203720
1      0.208918
2      0.129920
3      0.138855
4      0.092020
         ...   
718    0.086128
719    0.180740
720    0.165792
721    0.264821
722    0.223006
Name: min_cos_centroid_similarities, Length: 722, dtype: float64

## Maximal distance between the centroid and each point (2x)

In [844]:
training_data["max_eucl_centroid_distances"] = training_data["eucl_centroid_distances"].apply(np.max)
training_data["max_eucl_centroid_distances"]

0      1.083884
1      1.066734
2      1.096784
3      1.122352
4      1.170515
         ...   
718    1.140383
719    1.098227
720    1.083885
721    1.040934
722    1.077300
Name: max_eucl_centroid_distances, Length: 722, dtype: float64

In [845]:
training_data["max_cos_centroid_distances"] = training_data["cos_centroid_distances"].apply(np.max)
training_data["max_cos_centroid_distances"]

0      0.796280
1      0.791082
2      0.870080
3      0.861145
4      0.907980
         ...   
718    0.913872
719    0.819260
720    0.834208
721    0.735179
722    0.776994
Name: max_cos_centroid_distances, Length: 722, dtype: float64

In [846]:
training_data["max_cos_centroid_similarities"] = training_data["cos_centroid_similarities"].apply(np.max)
training_data["max_cos_centroid_similarities"]

0      0.787180
1      0.729358
2      0.702259
3      0.782002
4      0.795780
         ...   
718    0.725455
719    0.781997
720    0.710211
721    0.727465
722    0.802151
Name: max_cos_centroid_similarities, Length: 722, dtype: float64

## Index (minimumdistance/maximumdistance) (2x)

In [847]:
training_data["eucl_centroid_min_max"] = training_data.apply(lambda x: x.min_eucl_centroid_distances / x.max_eucl_centroid_distances if abs(x.max_eucl_centroid_distances) > 0 else 0, axis = 1)
training_data["eucl_centroid_min_max"]

0      0.579380
1      0.647402
2      0.655923
3      0.564701
4      0.522859
         ...   
718    0.608069
719    0.576724
720    0.655348
721    0.662646
722    0.565459
Name: eucl_centroid_min_max, Length: 722, dtype: float64

In [848]:
# uses distances
#training_data["cos_centroid_min_max"] = training_data.apply(lambda x: x.min_cos_centroid_distances / x.max_cos_centroid_distances, axis = 1)
#training_data["cos_centroid_min_max"]

In [849]:
training_data["cos_centroid_min_max"] = training_data.apply(lambda x: x.min_cos_centroid_similarities / x.max_cos_centroid_similarities if abs(x.max_cos_centroid_similarities) > 0 else 0, axis = 1)
training_data["cos_centroid_min_max"]

0      0.258798
1      0.286441
2      0.185003
3      0.177563
4      0.115634
         ...   
718    0.118723
719    0.231126
720    0.233441
721    0.364033
722    0.278010
Name: cos_centroid_min_max, Length: 722, dtype: float64

## Standard distance

In [850]:
training_data["standard_distance"] = training_data.apply(lambda x: np.sqrt(sum([sum(np.subtract(vector, x.TFIDF_centroid)**2) for vector in x.TFIDF_vector])) / len(x.TFIDF_vector), axis = 1)
training_data["standard_distance"]

0      0.090155
1      0.086912
2      0.085868
3      0.087804
4      0.085726
         ...   
718    0.082288
719    0.099277
720    0.085987
721    0.099010
722    0.105616
Name: standard_distance, Length: 722, dtype: float64

## Relative distance

In [851]:
training_data["eucl_relative_distance"] = training_data.apply(lambda x: x.standard_distance / x.max_eucl_centroid_distances if abs(x.max_eucl_centroid_distances) > 0 else 0, axis = 1)
training_data["eucl_relative_distance"]

0      0.083177
1      0.081475
2      0.078291
3      0.078232
4      0.073238
         ...   
718    0.072159
719    0.090397
720    0.079332
721    0.095116
722    0.098037
Name: eucl_relative_distance, Length: 722, dtype: float64

In [852]:
training_data["cos_relative_distance"] = training_data.apply(lambda x: x.standard_distance / x.max_cos_centroid_distances if abs(x.max_cos_centroid_distances) > 0 else 0, axis = 1)
training_data["cos_relative_distance"]

0      0.113220
1      0.109865
2      0.098690
3      0.101962
4      0.094414
         ...   
718    0.090044
719    0.121179
720    0.103077
721    0.134674
722    0.135929
Name: cos_relative_distance, Length: 722, dtype: float64

## Determinant of the distance matrix

In [853]:
training_data["det_eucl_dist_matrix"] = training_data["euclidian_distance_matrix"].apply(np.linalg.det)
training_data["det_eucl_dist_matrix"]

0     -4.476404e-22
1      1.701262e-24
2      1.425617e-28
3     -4.205273e-23
4     -2.971179e-23
           ...     
718    5.194272e-27
719   -8.136834e-19
720   -1.450856e-24
721   -9.458844e-19
722   -6.220687e-14
Name: det_eucl_dist_matrix, Length: 722, dtype: float64

In [854]:
training_data["det_cos_dist_matrix"] = training_data["cosine_distance_matrix"].apply(np.linalg.det)
training_data["det_cos_dist_matrix"]

0      -8.782959e-86
1       5.848917e-96
2      3.988340e-110
3      -9.181403e-90
4      -3.234747e-88
           ...      
718    8.421853e-106
719    -6.137569e-71
720    -3.702463e-98
721    -9.150974e-72
722    -1.901672e-54
Name: det_cos_dist_matrix, Length: 722, dtype: float64

In [855]:
training_data["det_cos_similarity_matrix"] = training_data["cosine_similarity_matrix"].apply(np.linalg.det)
training_data["det_cos_similarity_matrix"]

0       1.857575e-86
1       1.386964e-96
2      4.384316e-111
3       1.826199e-90
4       8.858751e-89
           ...      
718    2.157336e-106
719     1.773205e-71
720     8.837047e-99
721     2.657527e-72
722     7.421679e-55
Name: det_cos_similarity_matrix, Length: 722, dtype: float64

# Spatial autocorrelation

## Morans I

https://stackoverflow.com/questions/45839957/python-how-do-i-compute-interactive-spatial-autocorrelation-moran-i-using-py

In [857]:
training_data["weights_matrix"] = training_data["TFIDF_vector"].apply(lambda vectors: kneighbors_graph(vectors, 2, mode='connectivity', include_self = True).toarray())
training_data["weights_matrix"].iloc[0]

array([[1., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 1., 1.]])

In [858]:
training_data["sum_weights_matrix"] = training_data["weights_matrix"].apply(lambda vec: int(sum(sum(vec))))
training_data["sum_weights_matrix"].iloc[0]

136

In [859]:
training_data["nr_dimensions"] = training_data["TFIDF_vector"].apply(lambda vectors: vectors.shape[1])
training_data["nr_dimensions"].iloc[2]

305

In [860]:
training_data["morans_i_denom"] = training_data.apply(lambda x: sum([np.dot(np.subtract(vector, x.TFIDF_centroid), np.subtract(vector, x.TFIDF_centroid)) for vector in x.TFIDF_vector]), axis = 1)
training_data["morans_i_denom"]

0      37.583236
1      47.142577
2      55.809074
3      39.966356
4      33.981227
         ...    
718    51.252608
719    30.908090
720    52.170794
721    32.977068
722    25.700378
Name: morans_i_denom, Length: 722, dtype: float64

In [861]:
def get_morans_i(vec):
    """
    Result: positive -> positive spacial autocorrelation (clustering of neighbouring points), Negative --> opposite of positive & Zero -> spacial randomness
    """

    part1 = ((vec.nr_thesis_parts + 1) / vec.sum_weights_matrix) * (1 / vec.nr_dimensions)
    numerator_sum = []

    # loop over i points
    for i, vector in enumerate(vec.TFIDF_vector):

        numerator = []

        # loop over j points
        for j, vector2 in enumerate(vec.TFIDF_vector):
            
            if i != j:
                # wij * Di * Dj
                numerator.append(np.array([vec.weights_matrix[i][i], vec.weights_matrix[i][j]]) * np.dot(np.subtract(vector, vec.TFIDF_centroid), np.subtract(vector2, vec.TFIDF_centroid)))
        
        # sum of j
        numerator_sum.append(sum(numerator))

    # sum over all dimensions and sum of i
    part2 = sum(sum(numerator_sum) / vec.morans_i_denom)
    return part1 * part2


training_data["morans_i"] = training_data.apply(get_morans_i, axis = 1)
training_data["morans_i"]

0     -0.000170
1     -0.000132
2     -0.000117
3     -0.000192
4     -0.000266
         ...   
718   -0.000128
719   -0.000206
720   -0.000116
721   -0.000193
722   -0.000262
Name: morans_i, Length: 722, dtype: float64

## Geary’s C

In [862]:
def get_gearys_c(vec):
    """
    Result: 
    """
    # nr_thesis parts is already -1
    part1 = (np.array(vec.nr_thesis_parts) / 2) * (1 / np.array(vec.nr_dimensions))
    numerator_sum = []
    denominator_sum = []

    # loop over i points
    for i, vector in enumerate(vec.TFIDF_vector):

        numerator = []
        denominator = []

        # loop over j points
        for j, vector2 in enumerate(vec.TFIDF_vector):
            
            if i != j:
                numerator.append(np.array([vec.weights_matrix[i][i], vec.weights_matrix[i][j]]) * np.dot(np.subtract(vector, vector2), np.subtract(vector, vector2)))
                denominator.append(np.array([vec.weights_matrix[i][i], vec.weights_matrix[i][j]]) * np.dot(np.subtract(vector, vec.TFIDF_centroid), np.subtract(vector, vec.TFIDF_centroid)))
        
        # sum of j points
        numerator_sum.append(sum(numerator))
        denominator_sum.append(sum(denominator))
    
    # sum of i points
    num = np.array(sum(numerator_sum))
    denom = np.array(sum(denominator_sum))

    # outer sum over all dimensions
    part2 = sum(num / denom)
    return part1 * part2


training_data["gearys_c"] = training_data.apply(get_gearys_c, axis = 1)
training_data["gearys_c"]

0      0.304821
1      0.290923
2      0.307679
3      0.325623
4      0.389794
         ...   
718    0.293233
719    0.296006
720    0.260629
721    0.272364
722    0.246744
Name: gearys_c, Length: 722, dtype: float64

In [863]:
training_data["TFIDF_vector"].iloc[0]

array([[0.        , 0.13821339, 0.11577656, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.1191529 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.1207373 , ..., 0.        , 0.        ,
        0.05713795],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.16357415,
        0.18443422],
       [0.        , 0.        , 0.        , ..., 0.        , 0.19413428,
        0.21889158],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [864]:
np.array([1,2,1]) * np.array([2,2,2])

array([2, 4, 2])

## Gettis’s G

In [865]:
def get_weighting(x, i, j):
    """
    A weighting function wij(d) is used to assign binary weights to every pair of points, where wij(d) = 1, if i and j are within distance d and wij(d) = 0.
    Check in the distance matrix is i and j are within distance or not-
    """

    d = x.avg_eucl_dist
    distance = x.euclidian_distance_matrix[i][j]
    
    if distance <= d:
        return 1
    else:
        return 0

In [866]:
def get_gettis_g(vec):
    """
    d is the average distance between any two points in semantic space
    """
    # nr_thesis parts is already -1
    part1 = (1 / np.array(vec.nr_thesis_parts + 1))

    numerator_sum = []
    denominator_sum = []

    # loop over i points
    for i, vector in enumerate(vec.TFIDF_vector):

        numerator = []
        denominator = []

        # loop over j points
        for j, vector2 in enumerate(vec.TFIDF_vector):
            
            if i != j:

                d = get_weighting(vec, i, j)
                
                numerator.append(d * vector * vector2)
                denominator.append(vector * vector2) 
                
        
        # sum of j points
        numerator_sum.append(sum(numerator))
        denominator_sum.append(sum(denominator))
        

    # sum over i points and divide
    part2 = sum(sum(numerator_sum)) / sum(sum(denominator_sum))

    # part2 outer sum over all dimensions already done
    return part1 * part2


training_data["gettis_g"] = training_data.apply(get_gettis_g, axis = 1)
training_data["gettis_g"]

0      0.007702
1      0.006433
2      0.006090
3      0.007084
4      0.007050
         ...   
718    0.005433
719    0.010910
720    0.005894
721    0.010456
722    0.013335
Name: gettis_g, Length: 722, dtype: float64

# Set features of short essays to 0

In [867]:
# columns to fill with 0
col = ['min_neighbouring_eucl_dist','min_neighbouring_cos_similarity','max_neighbouring_eucl_dist', 'max_neighbouring_cos_similarity', 'eucl_min_max_quotient','cos_min_max_quotient', 
'avg_neighbouring_eucl_dist','avg_neighbouring_cos_similarity', 'avg_eucl_centroid_distances','avg_cos_centroid_similarities', 'min_eucl_centroid_distances','min_cos_centroid_similarities', 
'max_eucl_centroid_distances','max_cos_centroid_similarities', 'eucl_centroid_min_max', 'cos_centroid_min_max', 'avg_max_diff', 'standard_distance', 'morans_i', 'gearys_c', 'gettis_g',
'cos_clark_evan_dist_nn', 'eucl_clark_evan_dist_nn','avg_eucl_cumulative_freq_distribution','avg_cos_cumulative_freq_distribution', 'eucl_relative_distance','cos_relative_distance', 
'det_eucl_dist_matrix', "det_cos_similarity_matrix", "avg_cos_similarity_nn", "avg_eucl_dist_nn",

'avg_eucl_dist', 'avg_cos_similarity', 'max_eucl_dist', 'max_cos_similarity'
]

save_short_essays[col] = 0

In [868]:
save_short_essays['avg_cos_centroid_similarities'].value_counts()

0    1
Name: avg_cos_centroid_similarities, dtype: int64

In [869]:
feature_cols = [
    
'grade',

## preprocessed text
#'normalised_docs', 'word_tok', 'sentence_tok', 'stemmed_word_token', 'stemmed_no_stopwords', 
       
## Lexical sophistication
'nr_stopwords', 'avg_sentence_len_words', 'unique_words', 'word_count', 'char_count', 'avg_word_len', 'nr_long_words', 'nr_short_words',
'most_freq_word_length', 'nr_sentences', 'nr_long_sentences','nr_short_sentences', 'most_freq_sentence_length', 

## Readability measures
'dale_chall_readability', 'gunning_fox_index', 'flesh_reading_ease', 'flesh_kincaid_grade_level','automated_readability_index', 'SMOG', 'LIX', 'OVIX', 'nominal_ratio',

## Lexical diversity
'TTR', 'guirauds_index', 'yules_k', 'd_estimate', 'hapax_legomena', 'advanced_guiraud',

## Grammar
'nr_unique_pos', 'avg_tree_height', 'nr_grammar_errors', "correct_verb_form",

## POS tag
'comparative_adj', 'superlative_adj', 'modal_aux', 'participle', 'infinitive_marker', 'verb_baseform', 'verb_past_tense', 
# coordinating conjunction
'CCONJ', 
# subordinating conjunction
"SCONJ", 
# numeral
'NUM',
# determiner
'DET', 
# preposition
'ADP',
# adjektiv
'ADJ', 
# adverb 
'ADV', 
# pronoun
"PRON",
# proper noun
'PROPN', 
# common noun
'NOUN',

## Mechanics
'nr_spelling_errors', 'nr_capitalization_errors', 'nr_punctuation_errors',

## Content
'grade_as_feature','avg_cosine_similariy_high_grade', 'pattern_cosine', 'weighted_cosine',

#### Coherence ####

## Basic coherence measures
# Average distance between neighbouring points 
'avg_neighbouring_eucl_dist', 'avg_neighbouring_cos_similarity', 
# Minimum distance between neighbouring points 
'min_neighbouring_eucl_dist','min_neighbouring_cos_similarity',
# Maximum distance between neighbouring points 
'max_neighbouring_eucl_dist', 'max_neighbouring_cos_similarity', 
# Index (minimum distance/maximum distance) 
'eucl_min_max_quotient','cos_min_max_quotient',
# Average distance between any two points
"avg_eucl_dist", "avg_cos_similarity",
# Maximum difference between any two points
"max_eucl_dist", "max_cos_similarity",
# Clark’s and Evans’ distance to nearest neighbour
'eucl_clark_evan_dist_nn', # 'cos_clark_evan_dist_nn'
# Average distance to nearest neighbour
"avg_eucl_dist_nn", #"avg_cos_similarity_nn",
# Cumulative frequency distribution
'avg_eucl_cumulative_freq_distribution',#'avg_cos_cumulative_freq_distribution',  

## spatial data analysis
# Average distance between points and centroid 
'avg_eucl_centroid_distances','avg_cos_centroid_similarities', 
# Minimum distance between points and centroid
'min_eucl_centroid_distances','min_cos_centroid_similarities',
# Maximum distance between points and centroid 
'max_eucl_centroid_distances','max_cos_centroid_similarities', 
# Index (minimum distance/maximum distance) 
'eucl_centroid_min_max', 'cos_centroid_min_max',
# Standard distance
'standard_distance',
# Relative distance
'eucl_relative_distance',#'cos_relative_distance', 
# Determinant of distance matrix
'det_eucl_dist_matrix', #'det_cos_similarity_matrix'

## spatial autocorrelation
'morans_i', 'gearys_c', 'gettis_g',

]

training_data[feature_cols]

,grade,nr_stopwords,avg_sentence_len_words,unique_words,word_count,char_count,avg_word_len,nr_long_words,nr_short_words,most_freq_word_length,...,max_eucl_centroid_distances,max_cos_centroid_similarities,eucl_centroid_min_max,cos_centroid_min_max,standard_distance,eucl_relative_distance,det_eucl_dist_matrix,morans_i,gearys_c,gettis_g
0,34.0,281,15.0,259,679,2741,3.826215,335,344,3,...,1.083884,0.787180,0.579380,0.258798,0.090155,0.083177,-4.476404e-22,-0.000170,0.304821,0.007702
1,46.0,348,12.0,308,785,3180,3.945223,444,341,4,...,1.066734,0.729358,0.647402,0.286441,0.086912,0.081475,1.701262e-24,-0.000132,0.290923,0.006433
2,40.0,381,10.0,323,861,3547,3.979094,447,414,3,...,1.096784,0.702259,0.655923,0.185003,0.085868,0.078291,1.425617e-28,-0.000117,0.307679,0.006090
3,30.0,378,15.0,255,712,2692,3.693820,349,363,2,...,1.122352,0.782002,0.564701,0.177563,0.087804,0.078232,-4.205273e-23,-0.000192,0.325623,0.007084
4,26.0,370,11.0,203,671,2299,3.329359,313,358,4,...,1.170515,0.795780,0.522859,0.115634,0.085726,0.073238,-2.971179e-23,-0.000266,0.389794,0.007050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,35.0,399,9.0,342,862,3612,4.002320,296,566,3,...,1.140383,0.725455,0.608069,0.118723,0.082288,0.072159,5.194272e-27,-0.000128,0.293233,0.005433
719,32.0,259,9.0,218,556,2269,3.865108,304,252,4,...,1.098227,0.781997,0.576724,0.231126,0.099277,0.090397,-8.136834e-19,-0.000206,0.296006,0.010910
720,40.0,390,9.0,369,835,3780,4.294611,308,527,3,...,1.083885,0.710211,0.655348,0.233441,0.085987,0.079332,-1.450856e-24,-0.000116,0.260629,0.005894
721,40.0,272,7.0,250,576,2492,4.109375,188,388,3,...,1.040934,0.727465,0.662646,0.364033,0.099010,0.095116,-9.458844e-19,-0.000193,0.272364,0.010456


In [870]:
save_short_essays[feature_cols]

,grade,nr_stopwords,avg_sentence_len_words,unique_words,word_count,char_count,avg_word_len,nr_long_words,nr_short_words,most_freq_word_length,...,max_eucl_centroid_distances,max_cos_centroid_similarities,eucl_centroid_min_max,cos_centroid_min_max,standard_distance,eucl_relative_distance,det_eucl_dist_matrix,morans_i,gearys_c,gettis_g
713,10.0,1,4.0,5,5,18,3.6,2,3,2,...,0,0,0,0,0,0,0,0,0,0


In [871]:
feature_df = pd.concat([training_data[feature_cols], save_short_essays[feature_cols]], axis = 0)
feature_df

,grade,nr_stopwords,avg_sentence_len_words,unique_words,word_count,char_count,avg_word_len,nr_long_words,nr_short_words,most_freq_word_length,...,max_eucl_centroid_distances,max_cos_centroid_similarities,eucl_centroid_min_max,cos_centroid_min_max,standard_distance,eucl_relative_distance,det_eucl_dist_matrix,morans_i,gearys_c,gettis_g
0,34.0,281,15.0,259,679,2741,3.826215,335,344,3,...,1.083884,0.787180,0.579380,0.258798,0.090155,0.083177,-4.476404e-22,-0.000170,0.304821,0.007702
1,46.0,348,12.0,308,785,3180,3.945223,444,341,4,...,1.066734,0.729358,0.647402,0.286441,0.086912,0.081475,1.701262e-24,-0.000132,0.290923,0.006433
2,40.0,381,10.0,323,861,3547,3.979094,447,414,3,...,1.096784,0.702259,0.655923,0.185003,0.085868,0.078291,1.425617e-28,-0.000117,0.307679,0.006090
3,30.0,378,15.0,255,712,2692,3.693820,349,363,2,...,1.122352,0.782002,0.564701,0.177563,0.087804,0.078232,-4.205273e-23,-0.000192,0.325623,0.007084
4,26.0,370,11.0,203,671,2299,3.329359,313,358,4,...,1.170515,0.795780,0.522859,0.115634,0.085726,0.073238,-2.971179e-23,-0.000266,0.389794,0.007050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,32.0,259,9.0,218,556,2269,3.865108,304,252,4,...,1.098227,0.781997,0.576724,0.231126,0.099277,0.090397,-8.136834e-19,-0.000206,0.296006,0.010910
720,40.0,390,9.0,369,835,3780,4.294611,308,527,3,...,1.083885,0.710211,0.655348,0.233441,0.085987,0.079332,-1.450856e-24,-0.000116,0.260629,0.005894
721,40.0,272,7.0,250,576,2492,4.109375,188,388,3,...,1.040934,0.727465,0.662646,0.364033,0.099010,0.095116,-9.458844e-19,-0.000193,0.272364,0.010456
722,40.0,208,8.0,232,475,2059,4.181053,163,312,4,...,1.077300,0.802151,0.565459,0.278010,0.105616,0.098037,-6.220687e-14,-0.000262,0.246744,0.013335


# Save dataframe

In [873]:
feature_df.to_csv("csv_files/features_2_training_set_8.csv", index = False)